# ⚙️ Preprocesamiento de Datos - PAKDD 2010 Credit Risk

Este notebook aplica el pipeline de preprocesamiento definido en 
`src/features/build_features.py`, valida su funcionamiento, 
realiza la división Train/Test y guarda el pipeline entrenado 
para su uso en el notebook `03_Training.ipynb`.

---

## 🎯 Objetivos:
- Cargar el dataset intermedio con encabezados correctos  
- Separar variables predictoras (X) y variable objetivo (y)  
- Dividir los datos en train/test y guardarlos en `data/processed/`  
- Entrenar el pipeline de preprocesamiento (`BaseCleaner + ColumnTransformer`)  
- Guardar el pipeline entrenado en `models/preprocessing_pipeline.joblib`


## 📚 Importar Librerías

In [ ]:
import pandas as pd
import joblib
import sys
import os
# Asegura que Python pueda encontrar el módulo 'src'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.features.build_features import build_preprocessing_pipeline
from src.utils.split import split_and_save

pd.set_option('display.max_columns', 100)
pd.set_option('display.precision', 3)

print("✅ Librerías importadas correctamente.")

✅ Librerías importadas correctamente.


## 📁 Cargar Datos Intermedios

In [2]:
DATA_PATH = "../data/interim/train_clean_headers.parquet"
TARGET_COL = "TARGET_LABEL_BAD=1"

print(f"📥 Cargando dataset desde: {DATA_PATH}")

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"❌ No se encontró el archivo: {DATA_PATH}")

df = pd.read_parquet(DATA_PATH)

print(f"✅ Dataset cargado correctamente.")
print(f"📐 Dimensiones: {df.shape[0]:,} filas x {df.shape[1]} columnas")
display(df.head(3))

📥 Cargando dataset desde: ../data/interim/train_clean_headers.parquet
✅ Dataset cargado correctamente.
📐 Dimensiones: 50,000 filas x 54 columnas


,ID_CLIENT,CLERK_TYPE,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,QUANT_ADDITIONAL_CARDS,POSTAL_ADDRESS_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,EDUCATION_LEVEL,STATE_OF_BIRTH,CITY_OF_BIRTH,NACIONALITY,RESIDENCIAL_STATE,RESIDENCIAL_CITY,RESIDENCIAL_BOROUGH,FLAG_RESIDENCIAL_PHONE,RESIDENCIAL_PHONE_AREA_CODE,RESIDENCE_TYPE,MONTHS_IN_RESIDENCE,FLAG_MOBILE_PHONE,FLAG_EMAIL,PERSONAL_MONTHLY_INCOME,OTHER_INCOMES,FLAG_VISA,FLAG_MASTERCARD,FLAG_DINERS,FLAG_AMERICAN_EXPRESS,FLAG_OTHER_CARDS,QUANT_BANKING_ACCOUNTS,QUANT_SPECIAL_BANKING_ACCOUNTS,PERSONAL_ASSETS_VALUE,QUANT_CARS,COMPANY,PROFESSIONAL_STATE,PROFESSIONAL_CITY,PROFESSIONAL_BOROUGH,FLAG_PROFESSIONAL_PHONE,PROFESSIONAL_PHONE_AREA_CODE,MONTHS_IN_THE_JOB,PROFESSION_CODE,OCCUPATION_TYPE,MATE_PROFESSION_CODE,MATE_EDUCATION_LEVEL,FLAG_HOME_ADDRESS_DOCUMENT,FLAG_RG,FLAG_CPF,FLAG_INCOME_PROOF,PRODUCT,FLAG_ACSP_RECORD,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3,TARGET_LABEL_BAD=1
0,1,C,5,Web,0,1,F,6,1,0,RN,Assu,1,RN,Santana do Matos,Centro,Y,105,1.0,15.0,N,1,900.0,0.0,1,1,0,0,0,0,0,0.0,0,N,,None,None,N,,0,9.0,4.0,NaN,NaN,0,0,0,0,1,N,32,595,595,1
1,2,C,15,Carga,0,1,F,2,0,0,RJ,rio de janeiro,1,RJ,RIO DE JANEIRO,CAMPO GRANDE,Y,20,1.0,1.0,N,1,750.0,0.0,0,0,0,0,0,0,0,0.0,0,Y,,None,None,N,,0,11.0,4.0,11.0,NaN,0,0,0,0,1,N,34,230,230,1
2,3,C,5,Web,0,1,F,2,0,0,RN,GARANHUNS,1,RN,Parnamirim,Boa Esperanca,Y,105,1.0,NaN,N,1,500.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,None,None,N,,0,11.0,NaN,NaN,NaN,0,0,0,0,1,N,27,591,591,0


## ✂️ División en Train/Test y Guardado

In [3]:
print("🔧 Separando variables predictoras (X) y target (y)...")

X_train, X_test, y_train, y_test = split_and_save(df, TARGET_COL)

print("✅ División completada y datasets guardados en data/processed/")

🔧 Separando variables predictoras (X) y target (y)...
✅ Guardado en ../data/processed/ (train: (40000, 53), test: (10000, 53))
✅ División completada y datasets guardados en data/processed/


## 🧩 Crear y Entrenar el Pipeline de Preprocesamiento

In [4]:
from time import time
t0 = time()

print("⚙️ Construyendo pipeline...")
pipeline = build_preprocessing_pipeline()

print("🚀 Entrenando pipeline con los datos de entrenamiento...")
X_train_processed = pipeline.fit_transform(X_train, y_train)

print(f"✅ Pipeline entrenado correctamente en {time() - t0:.2f} segundos.")
print(f"📏 Forma original: {X_train.shape} → Transformada: {X_train_processed.shape}")

⚙️ Construyendo pipeline...
🚀 Entrenando pipeline con los datos de entrenamiento...
✅ Pipeline entrenado correctamente en 0.44 segundos.
📏 Forma original: (40000, 53) → Transformada: (40000, 151)


## 🔍 Verificación de las Features Generadas

In [5]:
try:
    feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
    print(f"🔤 Se generaron {len(feature_names)} columnas transformadas.")
    print("🔸 Ejemplo de columnas transformadas:")
    print(feature_names[:20])
except Exception as e:
    print(f"⚠️ No se pudieron obtener los nombres de columnas: {e}")

🔤 Se generaron 151 columnas transformadas.
🔸 Ejemplo de columnas transformadas:
['numeric_pipe__PERSONAL_ASSETS_VALUE'
 'numeric_pipe__QUANT_BANKING_ACCOUNTS'
 'numeric_pipe__QUANT_SPECIAL_BANKING_ACCOUNTS' 'numeric_pipe__AGE'
 'numeric_pipe__MONTHS_IN_RESIDENCE' 'numeric_pipe__MONTHS_IN_THE_JOB'
 'categorical_pipe__SEX_F' 'categorical_pipe__SEX_M'
 'categorical_pipe__SEX_N' 'categorical_pipe__PRODUCT_1'
 'categorical_pipe__PRODUCT_2' 'categorical_pipe__PRODUCT_7'
 'categorical_pipe__NACIONALITY_0' 'categorical_pipe__NACIONALITY_1'
 'categorical_pipe__NACIONALITY_2' 'categorical_pipe__MARITAL_STATUS_0'
 'categorical_pipe__MARITAL_STATUS_1' 'categorical_pipe__MARITAL_STATUS_2'
 'categorical_pipe__MARITAL_STATUS_3' 'categorical_pipe__MARITAL_STATUS_4']


## 💾 Guardar el Pipeline Entrenado

In [6]:

MODEL_DIR = "../models"
os.makedirs(MODEL_DIR, exist_ok=True)
PIPELINE_PATH = os.path.join(MODEL_DIR, "preprocessing_pipeline.joblib")

joblib.dump(pipeline, PIPELINE_PATH)

print(f"💾 Pipeline guardado exitosamente en: {PIPELINE_PATH}")

💾 Pipeline guardado exitosamente en: ../models\preprocessing_pipeline.joblib


## 🧾 Resumen del Proceso

In [7]:
print("✅ PREPROCESSING COMPLETADO EXITOSAMENTE ✅")
print(f"""
📊 Dataset inicial: {df.shape}
🔹 Train: {X_train.shape}
🔹 Test: {X_test.shape}
🧠 Pipeline entrenado y guardado en: {PIPELINE_PATH}
📁 Splits guardados en: ../data/processed/
""")

✅ PREPROCESSING COMPLETADO EXITOSAMENTE ✅

📊 Dataset inicial: (50000, 54)
🔹 Train: (40000, 53)
🔹 Test: (10000, 53)
🧠 Pipeline entrenado y guardado en: ../models\preprocessing_pipeline.joblib
📁 Splits guardados en: ../data/processed/

